<h1 align="center">Organization Info</h1> 

* Дедлайн **DD MM 2018 23:59** для всех групп.
* В качестве решения задания нужно прислать ноутбук с подробными комментариями (<span style='color:red'> без присланного решения результат контеста не будет засчитан </span>).
* <span style='color:red'>Название команды в контесте должно соответствовать шаблону: НомерГруппы_Имя_Фамилия, например, 594_Ivan_Ivanov</span>.

**Оформление дз**: 
- Присылайте выполненное задание на почту ``ml.course.mipt@gmail.com``
- Укажите тему письма в следующем формате ``ML2018_fall_<номер_группы>_<фамилия>``, к примеру -- ``ML2018_fall_495_ivanov``
- Выполненное дз сохраните в файл ``<фамилия>_<группа>_task<номер>.ipnb, к примеру`` -- ``ivanov_401_task7.ipnb``

**Вопросы**:
- Присылайте вопросы на почту ``ml.course.mipt@gmail.com``
- Укажите тему письма в следующем формате ``ML2018_fall Question <Содержание вопроса>``


--------
- **PS1:** Используются автоматические фильтры, и просто не найдем ваше дз, если вы неаккуратно его подпишите.
- **PS2:**  Просроченный дедлайн снижает максимальный вес задания по формуле, указнной на первом семинаре
- **PS3:** Допустимы исправление кода предложенного кода ниже, если вы считаете

<h1 align="center">Checking Questions</h1> 

**Вопрос 1**: Чем LSTM лучше/хуже чем обычная RNN?

**Ответ:**

Лучше: RNN имеет проблемы с градиентами (они могут затухать или наоборот взрывается), в отличии от LSTM, где это проблема решена с помощью forget gate. Это на самом деле, очень крутая фича.

Хуже: Главный минум относительно RNN - медленнее учится, а также сложнее (что плохо для понимания).

**Вопрос 2**:  Выпишите производную $\frac{d c_{n+1}}{d c_{k}}$ для LSTM http://colah.github.io/posts/2015-08-Understanding-LSTMs/, объясните формулу, когда производная затухает, когда взрывается?

**Ответ:**

**Вопрос 3**: Зачем нужен TBPTT почему BPTT плох?

**Ответ:** Проблема BPTT в градиентах (они могут взрываться). TBPTT решает эту проблему, для этого он и нужен.

**Вопрос 4**: Как комбинировать рекуррентные и сверточные сети, а главное зачем? Приведите несколько примеров реальных задач.

**Ответ:** Используются, например, в задаче генерации подписи к изображениям. Как комбинировать рекуррентные и сверточные сети: выход сверточной сети можно сувать на разные итерации рекуррентной. Зачем: когда мы хотим сувать не голые изображения, а какую-нибудь хорошую сеть.

**Вопрос 5**: Можно ли использовать сверточные сети для классификации текстов? Если нет обоснуйте :D, если да то как? как решить проблему с произвольной длинной входа?

**Ответ:** Нужно рассмотреть word embedding и работать с ними как с пикселями у изображений.

**Вопрос 6**: Attention - что это такое, где применяют и как? Приведите пример использования на какой-нибудь задаче

**Ответ:** Attention - это так называемый механизм внимания. Большинство NMT-систем работают, кодируя исходное предложение в вектор, используя повторяющуюся нейронную сеть, а затем декодируют предложение на основе этого вектора, также используя RNN. С механизмом внимания мы больше не пытаемся кодировать полное предложение источника в вектор фиксированной длины. Скорее, мы разрешаем декодеру «посещать» разные части исходного предложения на каждом этапе генерации вывода.

Пример использования в задача, которую мы решали ниже (в Part1 и Part2).

## Grading
* starting at zero points
* +2 for describing your iteration path in a report below (compare models).
* +2 for correct check questions
* +3 (7 total) for 99% accuracy with simple NMT model on __TEST__ dataset
* +3 (10 total) for 99% accuracy with attention NMT model on __TEST__ dataset
----
* tatoeba bonus for accuracy on __TEST__ dataset:
    * +2 for report
    * 60% (14 total)
    * 65% (16 total)
    * 70% (18 total)
    * 75% (20 total)
    
## Bonus points

Common ways to get bonus points are:
* Get higher score, obviously.
* Anything special about your NN. For example "A super-small/fast NN that gets 99%" gets a bonus.
* Any detailed analysis of the results. (attention maps, whatever)

---

In [0]:
# additional packages for this notebook

In [84]:
! pip install faker tqdm babel

## Task - translation

The machine translation is old and well-known field in natural language processing. From the 1950s scientists tried to create a model to automatically translate from say French to English. Nowadays it became possible and the attention mechanism takes great part in that. Here the example image with attention map for the neural machine translation of sample phrase:
<p align="center">
  <img src="http://d3kbpzbmcynnmx.cloudfront.net/wp-content/uploads/2015/12/Screen-Shot-2015-12-30-at-1.23.48-PM.png" width="400">
</p>

In our lab we will concentrate on much simplier task: we will translate from human readable date to machine readable one.

To do this we need to get one more concept - Sequence-to-Sequence language modeling.
The idea of such architecture is here:
<p aling="center">
<img src="./img/simple_nmt.jpg" width="400">
</p>

There is an Embeding layer at the bottom, the RNN in the middle and softmax as an output.

In [0]:
from keras.layers import Embedding, Bidirectional
from keras.layers.core import *
from keras.layers.recurrent import LSTM
from keras.models import *
from keras.layers.merge import Multiply
from keras.utils import to_categorical
from keras.layers import TimeDistributed

import keras.backend as K
import numpy as np

---

### Data

Now we need to generate data. It will be dates in different text formats and in fixed output format.

In [0]:
from faker import Faker
import random
from tqdm import tqdm
from babel.dates import format_date
import numpy as np

In [0]:
fake = Faker()

FORMATS = ['short',
           'medium',
           'long',
           'full',
           'd MMM YYY', 
           'd MMMM YYY',
           'dd MMM YYY',
           'd MMM, YYY',
           'd MMMM, YYY',
           'dd, MMM YYY',
           'd MM YY',
           'd MMMM YYY',
           'MMMM d YYY',
           'MMMM d, YYY',
           'dd.MM.YY']

# change this if you want it to work with another language
LOCALES = ['en_US']

In [0]:
def create_date():
    """
        Creates some fake dates 
        :returns: tuple containing human readable string, machine readable string, and date object
    """
    dt = fake.date_object()

    try:
        human_readable = format_date(dt, format=random.choice(FORMATS), locale=random.choice(LOCALES))

        case_change = random.choice([0,1,2])
        if case_change == 1:
            human_readable = human_readable.upper()
        elif case_change == 2:
            human_readable = human_readable.lower()
        # if case_change == 0, do nothing

        machine_readable = dt.isoformat()
    except AttributeError as e:
        return None, None, None

    return human_readable, machine_readable, dt

In [0]:
def create_dataset(n_examples):
    """
        Creates a dataset with n_examples and vocabularies
        :n_examples: the number of examples to generate
    """
    human_vocab = set()
    machine_vocab = set()
    dataset = []

    for i in tqdm(range(n_examples)):
        h, m, _ = create_date()
        if h is not None:
            dataset.append((h, m))
            human_vocab.update(tuple(h))
            machine_vocab.update(tuple(m))

    human = dict(zip(list(human_vocab) + ['<unk>', '<pad>'], 
                     list(range(len(human_vocab) + 2))))
    inv_machine = dict(enumerate(list(machine_vocab) + ['<unk>', '<pad>']))
    machine = {v:k for k,v in inv_machine.items()}
 
    return dataset, human, machine, inv_machine

In [0]:
def string_to_int(string, lenght, vocab):
    if len(string) > lenght:
        string = string[:lenght]
        
    rep = list(map(lambda x: vocab.get(x, '<unk>'), string))
    
    if len(string) < lenght:
        rep += [vocab['<pad>']] * (lenght - len(string))
    
    return rep

In [0]:
def int_to_string(ints, inv_vocab):
    return [inv_vocab[i] for i in ints]

Actually generating data:

In [92]:
fake.seed(42)
random.seed(42)
N = int(3e5)
dataset, human_vocab, machine_vocab, inv_machine_vocab = create_dataset(N)

100%|██████████| 300000/300000 [00:18<00:00, 16292.35it/s]


In [93]:
dataset[2]

('tuesday, september 14, 1971', '1971-09-14')

In [0]:
TIME_STEPS = 20

In [0]:
inputs, targets = zip(*dataset)
inputs = np.array([string_to_int(i, TIME_STEPS, human_vocab) for i in inputs])
targets = [string_to_int(t, TIME_STEPS, machine_vocab) for t in targets]
targets = np.array(list(map(lambda x: to_categorical(x, num_classes=len(machine_vocab)), targets)))

In [0]:
X_train, y_train, X_valid, y_valid, X_test, y_test = (
    inputs[:int(2e5)], targets[:int(2e5)], 
    inputs[int(2e5):-int(5e4)], targets[int(2e5):-int(5e4)],  
    inputs[-int(5e4):], targets[-int(5e4):], )

In [97]:
print(X_train.shape, y_train.shape)
print(len(human_vocab), len(machine_vocab))

(200000, 20) (200000, 20, 13)
60 13


---

### Part 1: Simple NMT

In [0]:
---

In [0]:
# :good-enouht:
ENCODER_UNITS = 64 # change me if u want
DECODER_UNITS = 32 # change me if u want
TIME_STEPS = 20 # change me if u want

In [0]:
# input - [bs; in_time_len]
# output - [bs; out_time_len]; out_time_len=10

def model_simple_nmt(in_chars, out_chars):
    # RNN encoder -> hidden representation -> RNN decoder
    inputs = Input(shape=(TIME_STEPS,))
    embedding = Embedding(input_dim=in_chars, output_dim=ENCODER_UNITS, input_length=TIME_STEPS)(inputs)
    lstm = Bidirectional(LSTM(DECODER_UNITS, return_sequences=True))(embedding)
    dense = Dense(out_chars, activation='softmax')(lstm)
    
    model = Model(input=[inputs], output=dense)
    return model

In [43]:
m = model_simple_nmt(len(human_vocab), len(machine_vocab))

m.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print(m.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 20)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 20, 64)            3840      
_________________________________________________________________
bidirectional_1 (Bidirection (None, 20, 64)            24832     
_________________________________________________________________
dense_1 (Dense)              (None, 20, 13)            845       
Total params: 29,517
Trainable params: 29,517
Non-trainable params: 0
_________________________________________________________________
None


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`
  if __name__ == '__main__':


In [50]:
m.fit(
    [X_train], y_train, 
    validation_data=(X_valid, y_valid),
    epochs=7, batch_size=64, 
    validation_split=0.1)

Train on 200000 samples, validate on 50000 samples
Epoch 1/7
 42240/200000 [=====>........................] - ETA: 4:15 - loss: 0.0431 - acc: 0.9864

113152/200000 [===============>..............] - ETA: 2:19 - loss: 0.0374 - acc: 0.9877

184960/200000 [==========================>...] - ETA: 24s - loss: 0.0351 - acc: 0.9883

200000/200000 [==============================] - 341s 2ms/step - loss: 0.0347 - acc: 0.9885 - val_loss: 0.0374 - val_acc: 0.9873
Epoch 2/7
 21440/200000 [==>...........................] - ETA: 4:45 - loss: 0.0306 - acc: 0.9896

 93376/200000 [=============>................] - ETA: 2:51 - loss: 0.0299 - acc: 0.9897

...

150656/200000 [=====================>........] - ETA: 1:18 - loss: 0.0240 - acc: 0.9912

200000/200000 [==============================] - 340s 2ms/step - loss: 0.0231 - acc: 0.9915 - val_loss: 0.0203 - val_acc: 0.9922
Epoch 7/7
  8768/200000 [>.............................] - ETA: 5:11 - loss: 0.0202 - acc: 0.9922

 81408/200000 [===========>..................] - ETA: 3:11 - loss: 0.0206 - acc: 0.9920

164224/200000 [=======================>......] - ETA: 57s - loss: 0.0208 - acc: 0.9920

200000/200000 [==============================] - 343s 2ms/step - loss: 0.0213 - acc: 0.9918 - val_loss: 0.0214 - val_acc: 0.9918


In [51]:
m.evaluate([X_test], y_test)

50000/50000 [==============================] - 44s 889us/step


[0.021406254270747303, 0.9917460008621216]

Lets check our model:

In [0]:
EXAMPLES = ['3 May 1979', '5 Apr 09', '20th February 2016', 'Wed 10 Jul 2007']

def run_example(model, input_vocabulary, inv_output_vocabulary, text):
    encoded = string_to_int(text, TIME_STEPS, input_vocabulary)
    prediction = model.predict(np.array([encoded]))
    prediction = np.argmax(prediction[0], axis=-1)
    return int_to_string(prediction, inv_output_vocabulary)

def run_examples(model, input_vocabulary, inv_output_vocabulary, examples=EXAMPLES):
    predicted = []
    for example in examples:
        predicted.append(''.join(run_example(model, input_vocabulary, inv_output_vocabulary, example)))
        print('input:', example)
        print('output:', predicted[-1])
    return predicted

In [53]:
run_examples(m, human_vocab, inv_machine_vocab)

input: 3 May 1979
output: 1979-05-03<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
input: 5 Apr 09
output: 2009-04-05<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
input: 20th February 2016
output: 2012-06-20<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
input: Wed 10 Jul 2007
output: 2007-06-00<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>


['1979-05-03<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>',
 '2009-04-05<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>',
 '2012-06-20<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>',
 '2007-06-00<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>']

In [54]:
---

SyntaxError: ignored

### Part 2: All u need is attention

Here we use more complex idea that simple seq2seq: we're adding two explicit parts of our network - encoder and decoder (which is applied attention on). The explanatory picture for this idea is below:
<p aling="center"><img src="https://i.stack.imgur.com/Zwsmz.png"></p>

The lower part of the network is encoding the input to some hidden intermediate representation and the upper part is decoing the hidвen represenataion into some readable output.

In [0]:
# :good-enouht:
ENCODER_UNITS = 32 # change me if u want
DECODER_UNITS = 32 # change me if u want
TIME_STEPS = 20 # change me if u want

In [0]:
def model_simple_nmt(in_chars, out_chars):
    encoder_input = Input(shape=(TIME_STEPS,))
    encoder_embedding = Embedding(input_dim=in_chars, output_dim=ENCODER_UNITS, input_length=TIME_STEPS)(encoder_input)
    encoder_LSTM = LSTM(ENCODER_UNITS,return_state = True)
    encoder_outputs, encoder_h, encoder_c = encoder_LSTM(encoder_embedding)
    encoder_states = [encoder_h, encoder_c]
    #Attention
    dense = Dense(20, activation='softmax')(encoder_outputs)
    rv = RepeatVector(20)(dense)
    
    decoder_LSTM = LSTM(DECODER_UNITS,return_sequences=True, return_state = True)
    decoder_out, _ , _ = decoder_LSTM(rv, initial_state=encoder_states)
    decoder_dense = Dense(out_chars, activation='softmax')
    decoder_out = decoder_dense(decoder_out)
    model = Model(inputs=[encoder_input],outputs=decoder_out)
    return model

In [115]:
m = model_simple_nmt(len(human_vocab), len(machine_vocab))

m.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print(m.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_27 (InputLayer)           (None, 20)           0                                            
__________________________________________________________________________________________________
embedding_18 (Embedding)        (None, 20, 32)       1920        input_27[0][0]                   
__________________________________________________________________________________________________
lstm_28 (LSTM)                  [(None, 32), (None,  8320        embedding_18[0][0]               
__________________________________________________________________________________________________
dense_16 (Dense)                (None, 20)           660         lstm_28[0][0]                    
__________________________________________________________________________________________________
repeat_vec

In [116]:
m.fit(
    [X_train], y_train, 
    validation_data=(X_valid, y_valid),
    epochs=5, batch_size=64, 
    validation_split=0.1)

Train on 200000 samples, validate on 50000 samples
Epoch 1/5
 41600/200000 [=====>........................] - ETA: 4:18 - loss: 0.9305 - acc: 0.7037

112768/200000 [===============>..............] - ETA: 2:20 - loss: 0.6360 - acc: 0.7829

183872/200000 [==========================>...] - ETA: 25s - loss: 0.4933 - acc: 0.8320

200000/200000 [==============================] - 339s 2ms/step - loss: 0.4695 - acc: 0.8402 - val_loss: 0.1878 - val_acc: 0.9376
Epoch 2/5
 21440/200000 [==>...........................] - ETA: 4:45 - loss: 0.1740 - acc: 0.9433

 92352/200000 [============>.................] - ETA: 2:51 - loss: 0.1434 - acc: 0.9531

...

156480/200000 [======================>.......] - ETA: 1:09 - loss: 0.0284 - acc: 0.9908

200000/200000 [==============================] - 341s 2ms/step - loss: 0.0274 - acc: 0.9910 - val_loss: 0.0222 - val_acc: 0.9921
Epoch 5/5
 10560/200000 [>.............................] - ETA: 5:04 - loss: 0.0234 - acc: 0.9918

 80960/200000 [===========>..................] - ETA: 3:10 - loss: 0.0220 - acc: 0.9921

154560/200000 [======================>.......] - ETA: 1:13 - loss: 0.0214 - acc: 0.9922

200000/200000 [==============================] - 343s 2ms/step - loss: 0.0209 - acc: 0.9923 - val_loss: 0.0184 - val_acc: 0.9928


In [117]:
m.evaluate([X_test], y_test)

50000/50000 [==============================] - 44s 886us/step


[0.01843271078594029, 0.9928049998092652]

In [0]:
---

## Report

* final architectures:

Part 1: input + embedding + bidirectional(LSTM) + dense

Part 2: input + embedding + LSTM + dense + repeat_vector + LSTM + dense

* comparison

Как и планировалось, скор у Part 2 (с attention) получился выше, даже учитывая, что у Part 1 было 12 эпох в сумме, а у Part 2 было 5 эпох. Лосс у Part 2, как мы видим, тоже получился меньше, чем у Part 1. Можно сделать вывод, что attention со своей функцией справляется.

* as well as training method and tricks

Как я получал модель Part 1: для начала я прочитал лекции и семинары, а также изучил статью https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html Part 1 описан в статье, как базовый случай и сразу стало понятно, что для его реализации нам понадобится LSTM, а также Dense. После изучания LSTM стало понятно, что нужна "кодировка" в лице embedding. Я добавил embedding. Запустил. Училось хорошо, но недостаточно быстро и на 10 эпохах только доходило до 0.9. Нужно было как-то увеличивать скор. После прочтения многих гайдов и статей я понял, что требуется добавить bidirectional. Он должен был помощь. И он помог. Для начала я запустил на 5 этохах. Скор был 0.98, после, я добавил еще 7 эпох (как позже оказалось, хватило бы в общей сложности 8 эпох) и модель доучилась до 0.99+.

Как я получал модель Part 2: Как известно из предыдущего абзаца, я прочитал статью. Из нее становится понятно, что базовый случай не прокатит и нужно делать общий. Я реализовал input + embedding + LSTM + dense и стал думать над "своим" attention. Прочитав еще раз лекцию, я понял, что нам нужно добавить слои dense(softmax) + repeat_vector. Это действительно будет похоже на обычный attention, так как мы будем фокусироваться на отдельных вещах, правда, будет делать это всего 1 раз (поэтому ожидать сильного буста не стоит), но в итоге небольшой буст мы получили. Так я получил итоговыю модель.
 

---

## Part 3*: tatoeba - real NMT

### Data

In [0]:
# dataset from http://www.manythings.org/anki/

In [0]:
! wget http://www.manythings.org/anki/rus-eng.zip

In [0]:
! unzip ./rus-eng.zip

In [0]:
with open("./rus.txt") as fin:
    data = fin.readlines()
data = list(map(lambda x: x.replace("\n", "").lower(), data))

In [0]:
len(data)

In [0]:
data = data[:int(1e5)]

In [0]:
len(data)

In [0]:
data[-1]

----

In [0]:
source = list(map(lambda x: x.split("\t")[0], data))
target = list(map(lambda x: x.split("\t")[1], data))

In [0]:
source_vocab = set("".join(source).strip())
target_vocab = set("".join(target).strip())

In [0]:
source_vocab = dict(zip(
    list(source_vocab) + ['<unk>', '<pad>'], 
    list(range(len(source_vocab) + 2))))
target_vocab = dict(zip(
    list(target_vocab) + ['<unk>', '<pad>'], 
    list(range(len(target_vocab) + 2))))
inv_target_vocab = dict(enumerate(list(target_vocab) + ['<unk>', '<pad>']))

In [0]:
TIME_STEPS = 32
ENCODER_UNITS = 256
DECODER_UNITS = 256

In [0]:
def model_simple_nmt_tatoeba(in_chars, out_chars):
    inputs = Input(shape=(TIME_STEPS,))
    
    # your code

    model = Model(input=[inputs], output=output)
    return model

In [0]:
m = model_attention_nmt(len(human_vocab), len(machine_vocab))

m.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print(m.summary())

In [0]:
inputs = np.array([string_to_int(i, TIME_STEPS, source_vocab) for i in source])
targets = [string_to_int(t, TIME_STEPS, target_vocab) for t in target]
targets = np.array(list(map(lambda x: to_categorical(x, num_classes=len(target_vocab)), targets)))

In [0]:
m.fit(
    [inputs], targets, 
    epochs=10, batch_size=64, 
    validation_split=0.1)

In [0]:
run_example(m, source_vocab, inv_target_vocab, 'hello')

### Tatoeba Report

* final architectures
* comparison
* as well as training method and tricks
